In [1]:
import pandas as pd
import numpy as np
import joblib as jb
from scipy.stats import spearmanr

In [2]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test, y_test = test.filter(regex=r'feature', axis=1), test['target_kazutsugi']

In [3]:
import glob


preds_train1 = glob.glob("./preds_train1/*.pkl.z")
preds_val1 = glob.glob("./preds_val1/*.pkl.z")

df_train1 = []
for p_name in preds_train1:
    p = jb.load(p_name)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_train1.append(p_df)
    
    
df_val1 = [] 
scores_val1 = dict()
for p_name in preds_val1:
    p = jb.load(p_name)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_val1.append(p_df)
    scores_val1[p_name] = spearmanr(y_val1, p_df).correlation
    
    
df_train1 = pd.concat(df_train1, axis=1)
df_val1 = pd.concat(df_val1, axis=1)

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [4]:
df_train1.shape

(178773, 208)

In [5]:
df_val1.shape

(180938, 208)

In [38]:
from sklearn.linear_model import LinearRegression, Ridge
#mdl = LinearRegression(normalize=True)
mdl = Ridge(alpha=1.)
mdl.fit(df_train1, y_train1)
p = mdl.predict(df_val1)
spearmanr(y_val1, p).correlation

0.03739169408417786

In [40]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
mdl = ExtraTreesRegressor(n_estimators=100, min_samples_leaf=100,random_state=0, n_jobs=-1)
mdl.fit(df_train1, y_train1)
p = mdl.predict(df_val1)
spearmanr(y_val1, p).correlation

0.030325668709151783

In [26]:
from lightgbm import LGBMRegressor
mdl = LGBMRegressor(n_estimators=200, min_samples_leaf=10, num_leaves=5, 
                    colsample_bytree=0.1,random_state=0, n_jobs=-1, learning_rate=1e-2)
mdl.fit(df_train1, y_train1)
p = mdl.predict(df_val1)
spearmanr(y_val1, p).correlation

0.0367347676525371

In [41]:
for p_name, score in scores_val1.items():
    if score > 0.035:
        print(p_name, score)

./preds_val1/lgbm_200_698_0.01.pkl.z 0.03556051361946144
./preds_val1/lgbm_130_369_0.009060562467991657.pkl.z 0.035934615261434086
./preds_val1/lgbm_165_766_0.01.pkl.z 0.035659178197050126


# Test

In [45]:
df_test = [] 
scores_test = dict()
preds_test = glob.glob("./preds_test/*.pkl.z")
for p_name in preds_test:
    p = jb.load(p_name)
    p_df = pd.DataFrame(p, columns=[p_name])
    df_test.append(p_df)
    scores_test[p_name] = spearmanr(y_test, p_df).correlation
    
    
df_test = pd.concat(df_test, axis=1)

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [46]:
from sklearn.linear_model import LinearRegression, Ridge
#mdl = LinearRegression(normalize=True)
mdl = Ridge(alpha=1.)
mdl.fit(df_train1, y_train1)

p = mdl.predict(df_test)
spearmanr(y_test, p).correlation
#test spearman = 0.0212

In [47]:
for p_name, score in scores_test.items():
    if score > 0.02:
        print(p_name, score)

./preds_test/lgbm_170_624_0.002423157857285798.pkl.z 0.022915128397784215
./preds_test/lgbm_200_698_0.01.pkl.z 0.024971680709669597
./preds_test/lgbm_161_521_0.004773968291550972.pkl.z 0.023222116556194217
./preds_test/lgbm_130_369_0.009060562467991657.pkl.z 0.023900739580672233
./preds_test/lgbm_165_766_0.01.pkl.z 0.02444210066589131
./preds_test/lgbm_145_582_0.003446459897310035.pkl.z 0.02194667732803529
./preds_test/lgbm_200_1000_0.01.pkl.z 0.02377815515130155
./preds_test/lgbm_97_812_0.0030197929882137525.pkl.z 0.022001348030679775
./preds_test/lgbm_119_844_0.007210171877207321.pkl.z 0.023887808340765624


In [ ]:
# fazer diário